In [53]:
import os
import pandas as pd
from tabulate import tabulate

# Path to the Excel file with values to match
values_excel_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

# Path to the folder with multiple Excel files
inbound_folder_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Inbound'
outbound_folder_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Outbound'

# Read values to match from the Excel file
values_df = pd.read_excel(values_excel_path)

# Extract the values from the relevant column (e.g., 'Scrip Code')
values_to_match = values_df['Scrip Code'].astype(str).str.lower().tolist()

# Get a list of Excel files in the inbound folder
inbound_files = [os.path.join(inbound_folder_path, file) for file in os.listdir(inbound_folder_path) if file.endswith('.xlsx')]

# Check if there are no files in the inbound folder
if not inbound_files:
    print("No files present in the inbound folder.")
else:
    # Dictionary to store results for each sheet name
    merged_results = {}

    # Iterate through each file
    for file_path in inbound_files:
        # Extract file date from the filename
        file_date = os.path.basename(file_path)[:8]

        # Read all sheets from the Excel file into a dictionary of DataFrames
        dfs = pd.read_excel(file_path, sheet_name=None)

        # Iterate through each sheet
        for sheet_name, df in dfs.items():
            # Convert the fourth column to lowercase and filter rows
            filtered_rows = df[df.iloc[:, 3].astype(str).str.lower().isin(map(str.lower, values_to_match))]

            # Drop the specified columns
            filtered_rows = filtered_rows.drop(columns=['Stock URL', 'ISIN', 'Stock Deals Page'])

            # Add a new column for file date
            filtered_rows['File Date'] = file_date

            # Check if the sheet name exists in the merged results dictionary
            if sheet_name not in merged_results:
                merged_results[sheet_name] = filtered_rows
            else:
                # Append the filtered rows to the existing sheet name
                merged_results[sheet_name] = pd.concat([merged_results[sheet_name], filtered_rows])

        # Move the file to the outbound folder after processing all sheets
        outbound_file_path = os.path.join(outbound_folder_path, os.path.basename(file_path))
        os.rename(file_path, outbound_file_path)
        print(f"File moved to: {outbound_file_path}")

    # Display merged results
    for sheet_name, merged_df in merged_results.items():
        # Display the sheet name
        print(f"\nSheet Name: {sheet_name}")

        # Check if there are any rows in the merged result
        if not merged_df.empty:
            # Sort the result based on 'Code' and 'File Date'
            sorted_rows = merged_df.sort_values(by=['Code', 'File Date'])

            # Display the sorted rows in a tabular format
            print(tabulate(sorted_rows, headers='keys', tablefmt='pretty', showindex=False))
        else:
            print("No data present.")


File moved to: C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Outbound\20231204-DailyDeals-Trendlyne.xlsx
File moved to: C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Outbound\20231227-DailyDeals-Trendlyne.xlsx

Sheet Name: Bulk Block Deals
+-----------------------------------+-----------+---------------------------------------------------+----------+-----------+----------+---------------------+---------------+------------+----------+-------------------+-----------+
|               Stock               |   Code    |                    Client Name                    | Exchange | Deal Type |  Action  |        Date         | Average Price |  Quantity  | Traded % | Last Traded Price | File Date |
+-----------------------------------+-----------+---------------------------------------------------+----------+-----------+----------+---------------------+---------------+------------+----------+-------------------+-----------+
|    Enbee Trade & Finance L